In [1]:
import random
import numpy as np
import requests
from config import weather_api_key
import time
from datetime import datetime
import pandas as pd

In [2]:
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs


In [3]:
coordinates = list(lat_lngs)

In [4]:
for coordinate in coordinates:
    print(coordinate[0],coordinate[1])

-12.758229566918018 20.675740934927006
-66.0669111265793 102.1968809229013
-15.264397300244667 151.65247143511226
-29.24361818171365 74.80473612110953
87.22558821288834 126.15114456895066
-78.19529585633717 -62.24392626984755
-21.58053856204387 -169.23149248605836
-18.75949553477109 -113.4424731135251
62.80359913381767 -112.42150541702144
57.026212353094536 -13.462751244444235
-10.106369222404695 -85.86687491123375
-53.316552706892296 81.17282392337091
-61.14959053261717 -155.05883538775228
39.410706623140584 -174.80336606308728
-41.57627446108647 44.58725961294303
-43.848279912997306 -5.834552154993332
40.78816998561635 -146.90400905944483
63.31373333381427 12.091183129212027
-67.20480688931096 67.1711527678784
-48.29335620099473 149.2672173557682
34.80201098229951 165.50393054593576
-17.754631882440833 -11.340640551872013
-25.91220015446352 158.66809677481405
65.22641650759005 -12.972179649440903
-49.459847673889975 -158.45521427225646
83.68347625838169 143.83551984478316
-27.4509502

-79.96613774844197 148.556916349166
-16.79831537419146 44.089570138203584
52.66300241175972 40.77595279919052
-75.42081977130992 -1.7499766608430605
8.71256400801829 64.00709771062324
-24.268962590978404 -47.21178610746907
-3.4696497317977872 -120.41627025697295
-37.039054695006065 149.65218258045962
58.66651698124957 45.932974165455164
-8.438597386863435 -120.14301920690434
-89.94139052530993 -32.92302516468388
-69.33085375190741 152.02978465786333
39.385931324871876 117.03575625071761
10.950830992373369 -173.44166123424878
35.80129741298619 -85.0006922600593
-55.337405179401365 -99.86056029361835
53.15982680290253 16.226239875681784
47.79803219063544 -102.7136941086456
20.938793818103065 23.681902021193594
48.13545949425557 80.1067790849927
65.49746812617943 163.57622524321448
-55.92108411854233 161.88124111635574
-12.295928423366675 -108.73537005412821
-21.657864410704832 65.01058209316929
78.0046263966291 19.08930152316475
-64.71172301973698 69.1029479745192
-79.67875378446271 -126

In [5]:
from citipy import citipy

In [6]:
cities = []
for coordinate in coordinates:
    city=citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    
    if city not in cities:
        cities.append(city)

len(cities)

765

In [7]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [8]:
city_url = url + "&q=" + cities[0].replace(" ","+")
requests.get(city_url).json()

{'coord': {'lon': 18.6071, 'lat': 48.2156},
 'weather': [{'id': 801,
   'main': 'Clouds',
   'description': 'few clouds',
   'icon': '02n'}],
 'base': 'stations',
 'main': {'temp': 56.5,
  'feels_like': 55.09,
  'temp_min': 56.5,
  'temp_max': 56.5,
  'pressure': 1022,
  'humidity': 69,
  'sea_level': 1022,
  'grnd_level': 1003},
 'visibility': 10000,
 'wind': {'speed': 5.68, 'deg': 92, 'gust': 6.85},
 'clouds': {'all': 11},
 'dt': 1667246890,
 'sys': {'type': 2,
  'id': 2003682,
  'country': 'SK',
  'sunrise': 1667194136,
  'sunset': 1667230158},
 'timezone': 3600,
 'id': 3059101,
 'name': 'Levice',
 'cod': 200}

In [9]:
city_data = []

print("Beginning Data Retrival     ")
print("----------------------------")

record_count = 1
set_count = 1

## Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_weather_description = city_weather["weather"][0]["description"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Description": city_weather_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrival     
----------------------------
Processing Record 1 of Set 1 | leua
{'coord': {'lon': 18.6071, 'lat': 48.2156}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'base': 'stations', 'main': {'temp': 56.5, 'feels_like': 55.09, 'temp_min': 56.5, 'temp_max': 56.5, 'pressure': 1022, 'humidity': 69, 'sea_level': 1022, 'grnd_level': 1003}, 'visibility': 10000, 'wind': {'speed': 5.68, 'deg': 92, 'gust': 6.85}, 'clouds': {'all': 11}, 'dt': 1667246890, 'sys': {'type': 2, 'id': 2003682, 'country': 'SK', 'sunrise': 1667194136, 'sunset': 1667230158}, 'timezone': 3600, 'id': 3059101, 'name': 'Levice', 'cod': 200}
Processing Record 2 of Set 1 | albany
{'coord': {'lon': -73.9662, 'lat': 42.6001}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 60.84, 'feels_like': 59.43, 'temp_min': 58.53, 'temp_max': 62.73, 'pressure': 1012, 'humidity': 60}, 'visibility': 

{'cod': '404', 'message': 'city not found'}
City not found. Skipping...
Processing Record 16 of Set 1 | saldanha
{'coord': {'lon': 17.9442, 'lat': -33.0117}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 62.91, 'feels_like': 61.57, 'temp_min': 62.91, 'temp_max': 62.91, 'pressure': 1016, 'humidity': 57, 'sea_level': 1016, 'grnd_level': 1013}, 'visibility': 10000, 'wind': {'speed': 17.87, 'deg': 151, 'gust': 31.43}, 'clouds': {'all': 5}, 'dt': 1667247016, 'sys': {'country': 'ZA', 'sunrise': 1667188208, 'sunset': 1667236404}, 'timezone': 7200, 'id': 3361934, 'name': 'Saldanha', 'cod': 200}
Processing Record 17 of Set 1 | kodiak
{'coord': {'lon': -152.4072, 'lat': 57.79}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 33.31, 'feels_like': 20.71, 'temp_min': 33.31, 'temp_max': 33.31, 'pressure': 1009, 'humidity': 34}, 'visibility': 10000, '

{'coord': {'lon': -62.9967, 'lat': -40.8135}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 61.05, 'feels_like': 57.99, 'temp_min': 61.05, 'temp_max': 61.05, 'pressure': 1029, 'humidity': 24, 'sea_level': 1029, 'grnd_level': 1028}, 'visibility': 10000, 'wind': {'speed': 13.13, 'deg': 165, 'gust': 14.09}, 'clouds': {'all': 0}, 'dt': 1667247023, 'sys': {'country': 'AR', 'sunrise': 1667206836, 'sunset': 1667256627}, 'timezone': -10800, 'id': 3832899, 'name': 'Viedma', 'cod': 200}
Processing Record 33 of Set 1 | hilo
{'coord': {'lon': -155.09, 'lat': 19.7297}, 'weather': [{'id': 502, 'main': 'Rain', 'description': 'heavy intensity rain', 'icon': '10d'}], 'base': 'stations', 'main': {'temp': 79.3, 'feels_like': 79.3, 'temp_min': 77.25, 'temp_max': 80.33, 'pressure': 1018, 'humidity': 82}, 'visibility': 10000, 'wind': {'speed': 6.91, 'deg': 340}, 'rain': {'1h': 4.83}, 'clouds': {'all': 75}, 'dt': 1667246859, 'sys': 

{'coord': {'lon': 140.8333, 'lat': 35.7333}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 50.59, 'feels_like': 48.54, 'temp_min': 50.4, 'temp_max': 54.21, 'pressure': 1024, 'humidity': 68, 'sea_level': 1024, 'grnd_level': 1023}, 'visibility': 10000, 'wind': {'speed': 12.62, 'deg': 37, 'gust': 14.36}, 'clouds': {'all': 100}, 'dt': 1667247030, 'sys': {'type': 1, 'id': 7955, 'country': 'JP', 'sunrise': 1667249887, 'sunset': 1667288536}, 'timezone': 32400, 'id': 2112802, 'name': 'Hasaki', 'cod': 200}
Processing Record 49 of Set 1 | vaitupu
{'cod': '404', 'message': 'city not found'}
City not found. Skipping...
Processing Record 50 of Set 1 | chimbote
{'coord': {'lon': -78.5783, 'lat': -9.0853}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 66.36, 'feels_like': 66.13, 'temp_min': 66.36, 'temp_max': 66.36, 'pressure': 1012, 

{'coord': {'lon': -78.1162, 'lat': 48.5669}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 52.18, 'feels_like': 50.29, 'temp_min': 52.18, 'temp_max': 52.18, 'pressure': 1009, 'humidity': 68, 'sea_level': 1009, 'grnd_level': 973}, 'visibility': 10000, 'wind': {'speed': 4.99, 'deg': 184, 'gust': 16.04}, 'clouds': {'all': 100}, 'dt': 1667247096, 'sys': {'type': 2, 'id': 2033122, 'country': 'CA', 'sunrise': 1667217422, 'sunset': 1667253298}, 'timezone': -14400, 'id': 5884588, 'name': 'Amos', 'cod': 200}
Processing Record 16 of Set 2 | kalmunai
{'coord': {'lon': 81.8167, 'lat': 7.4167}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'base': 'stations', 'main': {'temp': 78.13, 'feels_like': 79.65, 'temp_min': 78.13, 'temp_max': 78.13, 'pressure': 1008, 'humidity': 85, 'sea_level': 1008, 'grnd_level': 1007}, 'visibility': 10000, 'wind': {'speed': 6.49, 'deg': 226, 'gust':

...

{'coord': {'lon': 33.5306, 'lat': -24.6867}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 71.1, 'feels_like': 71.44, 'temp_min': 71.1, 'temp_max': 71.1, 'pressure': 1021, 'humidity': 75, 'sea_level': 1021, 'grnd_level': 1009}, 'visibility': 10000, 'wind': {'speed': 15.57, 'deg': 166, 'gust': 25.93}, 'clouds': {'all': 100}, 'dt': 1667248071, 'sys': {'country': 'MZ', 'sunrise': 1667185151, 'sunset': 1667231979}, 'timezone': 7200, 'id': 1049861, 'name': 'Chibuto', 'cod': 200}
Processing Record 50 of Set 14 | cedar falls
{'coord': {'lon': -92.4455, 'lat': 42.5278}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 65.12, 'feels_like': 63.07, 'temp_min': 63.21, 'temp_max': 68.22, 'pressure': 1010, 'humidity': 37}, 'visibility': 10000, 'wind': {'speed': 10.36, 'deg': 290}, 'clouds': {'all': 0}, 'dt': 1667248072, 'sys': {'type': 2, 'id':

{'coord': {'lon': 13.6856, 'lat': 11.5221}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 79.45, 'feels_like': 79.45, 'temp_min': 79.45, 'temp_max': 79.45, 'pressure': 1011, 'humidity': 23, 'sea_level': 1011, 'grnd_level': 974}, 'visibility': 10000, 'wind': {'speed': 8.32, 'deg': 349, 'gust': 19.35}, 'clouds': {'all': 7}, 'dt': 1667248138, 'sys': {'country': 'NG', 'sunrise': 1667192221, 'sunset': 1667234435}, 'timezone': 3600, 'id': 2347954, 'name': 'Bama', 'cod': 200}
Processing Record 17 of Set 15 | warqla
{'cod': '404', 'message': 'city not found'}
City not found. Skipping...
Processing Record 18 of Set 15 | aksehir
{'coord': {'lon': 31.4164, 'lat': 38.3575}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'base': 'stations', 'main': {'temp': 52.23, 'feels_like': 49.32, 'temp_min': 52.23, 'temp_max': 52.23, 'pressure': 1020, 'humidity': 46, 'sea_level': 1020, 'grnd_le

{'coord': {'lon': -5.5371, 'lat': 50.1186}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'base': 'stations', 'main': {'temp': 56.08, 'feels_like': 55.67, 'temp_min': 56.08, 'temp_max': 56.08, 'pressure': 997, 'humidity': 91, 'sea_level': 997, 'grnd_level': 995}, 'visibility': 10000, 'wind': {'speed': 25.66, 'deg': 269, 'gust': 30}, 'rain': {'1h': 0.52}, 'clouds': {'all': 100}, 'dt': 1667248144, 'sys': {'type': 1, 'id': 1393, 'country': 'GB', 'sunrise': 1667200202, 'sunset': 1667235680}, 'timezone': 0, 'id': 2640377, 'name': 'Penzance', 'cod': 200}
Processing Record 32 of Set 15 | delmas
{'coord': {'lon': -72.3028, 'lat': 18.5447}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 88.52, 'feels_like': 97.27, 'temp_min': 88.52, 'temp_max': 88.52, 'pressure': 1009, 'humidity': 62}, 'visibility': 10000, 'wind': {'speed': 5.75, 'deg': 310}, 'clouds': {'all': 75}, 'dt': 1667

{'coord': {'lon': 23.0471, 'lat': -34.0363}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 57.04, 'feels_like': 56.25, 'temp_min': 57.04, 'temp_max': 57.04, 'pressure': 1026, 'humidity': 81, 'sea_level': 1026, 'grnd_level': 1025}, 'visibility': 10000, 'wind': {'speed': 8.88, 'deg': 83, 'gust': 15.19}, 'clouds': {'all': 6}, 'dt': 1667248150, 'sys': {'country': 'ZA', 'sunrise': 1667186891, 'sunset': 1667235271}, 'timezone': 7200, 'id': 988698, 'name': 'Knysna', 'cod': 200}
Processing Record 46 of Set 15 | salalah
{'coord': {'lon': 54.0924, 'lat': 17.0151}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 78.89, 'feels_like': 78.89, 'temp_min': 78.89, 'temp_max': 78.89, 'pressure': 1015, 'humidity': 65}, 'visibility': 10000, 'wind': {'speed': 4.61, 'deg': 330}, 'clouds': {'all': 0}, 'dt': 1667248150, 'sys': {'type': 1, 'id': 7562, 'country'

{'coord': {'lon': -81.2719, 'lat': -4.5772}, 'weather': [{'id': 761, 'main': 'Dust', 'description': 'dust', 'icon': '50d'}], 'base': 'stations', 'main': {'temp': 70.56, 'feels_like': 70.5, 'temp_min': 70.56, 'temp_max': 70.56, 'pressure': 1009, 'humidity': 68}, 'visibility': 8000, 'wind': {'speed': 19.57, 'deg': 170}, 'clouds': {'all': 26}, 'dt': 1667248216, 'sys': {'type': 1, 'id': 8685, 'country': 'PE', 'sunrise': 1667214033, 'sunset': 1667258201}, 'timezone': -18000, 'id': 3691582, 'name': 'Talara', 'cod': 200}
Processing Record 10 of Set 16 | shellbrook
{'coord': {'lon': -106.4011, 'lat': 53.2168}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 45.63, 'feels_like': 40.26, 'temp_min': 45.63, 'temp_max': 45.63, 'pressure': 1003, 'humidity': 30, 'sea_level': 1003, 'grnd_level': 945}, 'visibility': 10000, 'wind': {'speed': 10.98, 'deg': 284, 'gust': 15.55}, 'clouds': {'all': 46}, 'dt': 1667248217, 'sys'

In [14]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,Leua,48.2156,18.6071,56.50,69,11,5.68,SK,few clouds
1,Albany,42.6001,-73.9662,62.73,60,98,1.99,US,overcast clouds
2,Samarai,-10.6167,150.6667,82.35,77,7,12.48,PG,clear sky
3,Bambous Virieux,-20.3428,57.7575,73.65,83,40,6.91,MU,light rain
4,Tiksi,71.6872,128.8694,5.25,95,99,8.01,RU,overcast clouds
